**TEXT ANALYSIS**


In [ ]:
#import libraries
import numpy as np
import pandas as pd
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
#Loading our dataset
data = pd.read_csv('/content/SPAM text message 20170820 - Data.csv')
data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
data.shape

(5572, 2)

In [ ]:
#install nltk

! pip install nltk==3.4.5


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Punkt: This tokenizer divides a text into a list of sentences by using an unsupervised algorithm to build a model for abbreviation words, collocations, and words that start sentences. It must be trained on a large collection of plain text in a target language before it can be used.

In [ ]:
#Converting the labels from strings to binary values for our classifier

data['Category'] = data.Category.map({'ham':0,'spam':1})

In [ ]:
#Removing punctuation

data["Message"]=data.Message.str.replace('[^\w\s]', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#Converting all characters to lower to lower case
data["Message"] = data.Message.map(lambda x:x.lower())

In [ ]:
!pip install pandarallel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

data['Message'] = data['Message'].parallel_apply(lambda x: nltk.word_tokenize(x))
#creating a new column called 'new_col' in our dataframe
#The column will contain tokenized data from the 'Message' column.

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
data.head()

,Category,Message
0,0,"[go, until, jurong, point, crazy, available, o..."
1,0,"[ok, lar, joking, wif, u, oni]"
2,1,"[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,0,"[u, dun, say, so, early, hor, u, c, already, t..."
4,0,"[nah, i, dont, think, he, goes, to, usf, he, l..."


In [ ]:
from nltk.stem import PorterStemmer

In [ ]:
#Performing word stemming
stemmer = PorterStemmer()

data['Message'] = data['Message'].apply(lambda x: [stemmer. stem(y) for y in x])

In [ ]:
data.head()

,Category,Message
0,0,"[go, until, jurong, point, crazi, avail, onli,..."
1,0,"[ok, lar, joke, wif, u, oni]"
2,1,"[free, entri, in, 2, a, wkli, comp, to, win, f..."
3,0,"[u, dun, say, so, earli, hor, u, c, alreadi, t..."
4,0,"[nah, i, dont, think, he, goe, to, usf, he, li..."


In [ ]:
#Feature extraction
#Feature extraction refers to the process of transforming raw data into numerical features that can be processed 
#while preserving the information in the original data set. 
#It yields better results than applying machine learning directly to the raw data.

from sklearn.feature_extraction.text import CountVectorizer

**Count Vectorizer:**  It is used to transform a given text into a vector on the basis of the frequency (count) of each word that occurs in the entire text.

In [ ]:
# This converts the list of words into space-separated strings
data['Message'] = data['Message'].apply(lambda x: ' '.join(x))

In [ ]:
count_vect = CountVectorizer()
counts = count_vect.fit_transform(data['Message'])
#This is gives us word count per message

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer().fit(counts)
counts = transformer.transform(counts)

#Using TFI-DF, Term Frequency Inverse Document Frequency
In information retrieval, tf–idf, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus. It is often used as a weighting factor in searches of information retrieval, text mining, and user modeling.

In [ ]:
#Training the model

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(counts, data['Category'], test_size=0.1, random_state=69)

In [ ]:
#Fitting our model
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB().fit(X_train,y_train)

In [ ]:
#Evaluating the model
y_pred = model.predict(X_test)
print(np.mean(y_pred == y_test))

#The model classifies text as spam or ham with 94.62% accuracy

0.946236559139785


<5572x8165 sparse matrix of type '<class 'numpy.float64'>'
	with 72430 stored elements in Compressed Sparse Row format>